# Etsy API - Analyzing popular printed fabrics

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import json
import progressbar
import numpy as np
import plotly.graph_objects as go

In [2]:
load_dotenv()
KEY = os.getenv("KEYSTRING")
SECRET = os.getenv("SECRET")

## Get all listings for a specific store

In [3]:
def get_store(name, check_cache=True):
    if check_cache:
        try:
            df = pd.read_csv(f'data/store_{name}.csv')
            return df
        except:
            pass
    r = dict(requests.get(f'https://openapi.etsy.com/v2/shops/{name}/listings/active?api_key={KEY}&limit=100').json())
    count = r['count']
    print(f"Found {count} listings for store '{name}'")
    df = pd.DataFrame(r['results'])
    if count > 100:
        query = [int(x*100) for x in list(np.arange(1,np.ceil(count/100)))]
        for q in progressbar.progressbar(query, redirect_stdout=True):
            r2 = dict(requests.get(f'https://openapi.etsy.com/v2/shops/{name}/listings/active?api_key={KEY}&limit=100&offset={q}').json())
            df2 = pd.DataFrame(r2['results'])
            df = df.append(df2)
    df.drop(columns=['description']).to_csv(f'data/store_{name}.csv', index=False)
    return df

In [4]:
ZZ = get_store('zeetzeet')

## Visualize the views/number of favorites for a specific store

In [49]:
def viz_favorites(DF, store=None, search=None, field=None):
    if store:
        title = f'Views / Favorites for Store: "{store}"'
    elif search and field:
        title = f'Views / Favorites for {field}: {", ".join(search)}'
    else: title = ''
    fig = go.Figure(data=go.Scatter(x=DF['views'], y=DF['num_favorers'], mode='markers', text=DF['title']))
    fig.update_layout(
        title=title,
        xaxis_title="Views",
        yaxis_title="Favorites",
        annotations = [dict(xref='paper',
                            yref='paper',
                            x=0.95, y=0.95,
                            showarrow=False,
                            text =f'Viewing {DF.shape[0]} listings')]
    )
    fig.write_html(f"viz/{title.replace(' ','_').replace('/','vs')}.html")
    return fig

In [50]:
viz_favorites(ZZ, store='zeetzeet').show()

## Get all listings with terms containing keywords

In [51]:
def get_listings(search, field, check_cache=True):
    if check_cache:
        try:
            df = pd.read_csv(f'data/{field}_{search}.csv')
            return df
        except:
            pass
    r = requests.get(f"https://openapi.etsy.com/v2/listings/active.js?{field}={'+'.join(search)}&api_key={KEY}&limit=100")
    d = dict(json.loads(r.text[5:-2]))
    count = d['count']
    print(f"Found {count} listings for {field} {search}")
    df = pd.DataFrame(d['results'])
    if count > 100:
        query = [int(x*100) for x in list(np.arange(1,np.ceil(count/100)))]
        for q in progressbar.progressbar(query, redirect_stdout=True):
            r2 = requests.get(f"https://openapi.etsy.com/v2/listings/active.js?{field}={'+'.join(search)}&api_key={KEY}&limit=100&offset={q}")
            d2 = dict(json.loads(r2.text[5:-2]))
            df2 = pd.DataFrame(d2['results'])
            df = df.append(df2)
    df.drop(columns=['description']).to_csv(f'data/{field}_{search}.csv', index=False)
    return df

In [52]:
def get_and_viz_listings(search, field):
    df = get_listings(search, field)
    fig = viz_favorites(df, search=search, field=field)
    return fig

In [53]:
get_and_viz_listings(['Alexander Henry','fabric'], 'tags')

In [54]:
get_and_viz_listings(['Alexander Henry','fabric'], 'keywords')